# qSDK VQE: Custom Ansatz Tutorial
The qSDK comes packaged with an implementation of several standard ansatz circuits for the user to take advantage of. In this tutorial, we'll explore how you can incorporate the built in VQESolver into your own workflow, by introducing a user-defined custom ansatz circuit. We'll base our work here on the *VQESolver* class, and take advantage of tools readily available through the *qSDK* and *agnostic_simulator* package. 

In [2]:
import numpy as np
from qsdk.electronic_structure_solvers.vqe_solver import VQESolver
from qsdk.toolboxes.ansatz_generator.ansatz import Ansatz
from qsdk.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit
from qsdk.toolboxes.qubit_mappings.mapping_transform import get_qubit_number
from agnostic_simulator import Circuit, Gate

## Hardware Efficient Ansatz
For our example, we're going to implement the so-called [Hardware Efficient Ansatz](https://arxiv.org/pdf/1704.05018.pdf) (HEA), developed by Kandala et al at IBM. In this ansatz, a circuit is constructed with repeated layers of a simple structure. Each layer consists of entangling gates (e.g. CNOT or CZ) which couple neighbouring qubits, followed by a series of Euler rotations carried out as single-qubit rotations $\mathrm{exp}\{i\theta_i^1 Z_i\}\mathrm{exp}\{i\theta_i^2 X_i\}\mathrm{exp}\{i\theta_i^3 Z_i\}$. We'll start by initializing our *Ansatz* class, and then fill in the functionality required to implement this ansatz in *VQESolver*.

To construct our HEA ansatz, we're going to make use of the three helper functions defined here. The first will go through a register of qubits and add a layer of Euler-rotations as prescribed above. The second adds two columns of alternating CNOT gates, establishing long-range entanglement. The third brings these together into a sequence of alternating entanglers and Euler rotations.

In [3]:
def EulerCircuit(n_qubits):
    """Construct a circuit applying an Euler Z-X-Z rotation to each qubit."""
    circuit = Circuit()
    for target in range(n_qubits):
        circuit.add_gate(Gate("RZ" , target, parameter=0.0, is_variational=True))
        circuit.add_gate(Gate("RX", target, parameter=0.0, is_variational=True))
        circuit.add_gate(Gate("RZ", target, parameter=0.0, is_variational=True))
    return circuit

def EntanglerCircuit(n_qubits):
    """Construct a circuit applying two columns of staggered CNOT gates to all qubits
     and their neighbours"""
    circuit = Circuit()
    for ii in range(n_qubits//2):
        circuit.add_gate(Gate("CNOT", control=2*ii, target=2*ii + 1))
    for ii in range(n_qubits//2 - 1):
        circuit.add_gate(Gate("CNOT", control=2*ii + 1, target=2*(ii+1)))
    return circuit

def HEACircuit(n_qubits, n_layers):
    """Construct a circuit consisting of alternating sequence of Euler rotations and entanglers"""
    circuit = EulerCircuit(n_qubits)
    for ii in range(n_layers):
        circuit += EntanglerCircuit(n_qubits)
        circuit += EulerCircuit(n_qubits)
    return circuit

## Ansatz Class
In the VQESolver, we are expecting an instance of an abstract Ansatz class, which will be responsible for constructing the variational circuit we use to minimize the energy of our problem. To build up our own Ansatz class, we'll require the following.

0. **__init__**: an initialization function to instantiate the class.
1. **set_var_params**: initialize the variational circuit parameters
2. **update_var_params**: update the parametric gates in the circuit
3. **prepare_reference_state**: get fixed circuit for initializing the reference, e.g. HF state.
4. **build_circuit**: instantiate the variational circuit object

Below, we're going to type out the entire class as we will use it. This is a lot of code in one place. So afterwards, we'll break it down into each of the relevant member methods.

In [4]:
class HEA(Ansatz):

    def __init__(self, n_spinorbitals, n_electrons, n_layers, mapping='jw'):

        self.n_spinorbitals = n_spinorbitals
        self.n_qubits = get_qubit_number(mapping, n_spinorbitals)
        self.n_electrons = n_electrons
        #number of layers of repeated entangler + Euler rotations
        self.n_layers = n_layers
        
        #specify fermion-to-qubit mapping (required for the initial reference state)
        self.mapping = mapping
        
        #Each layer has 3 variational parameters per qubit, and one non-variational entangler
        #There is an additional layer with no entangler.
        self.n_var_params = self.n_qubits * 3 * (self.n_layers + 1)

        self.var_params = None
        self.circuit = None

    def set_var_params(self, var_params=None):
        """Set initial variational parameter values"""
        if var_params is None:
            var_params = np.random.random(self.n_var_params)
        elif var_params.size != self.n_var_params:
            raise ValueError('Invalid number of parameters.')
        self.var_params = var_params
        return var_params

    def update_var_params(self, var_params):
        """Update variational parameters (done repeatedly during VQE)"""
        for param_index in range(self.n_var_params):
            self.circuit._variational_gates[param_index].parameter = var_params[param_index]
    
    def prepare_reference_state(self):
        """Prepare a circuit generating the HF reference state."""
        return get_reference_circuit(n_spinorbitals=self.n_spinorbitals, n_electrons=self.n_electrons,mapping=self.mapping)

    def build_circuit(self, var_params=None):
        """Construct the variational circuit to be used as our ansatz."""
        self.var_params = self.set_var_params(var_params)

        reference_state_circuit = self.prepare_reference_state()
        hea_circuit = HEACircuit(self.n_qubits, self.n_layers)

        if reference_state_circuit.size != 0:
            self.circuit = reference_state_circuit + hea_circuit
        else:
            self.circuit = hea_circuit
        return self.circuit

Very briefly, we'll go through the member methods required to construct an *Ansatz* class. These code blocks duplicate the code above. We emphasize here that these member functions can be really as simple as you like.

Let's start with *set_var_params*. We're going to do something very basic and just force this to be a random numpy array. We add some error handling in case the number of parameters is incompatible with the number of variational gates in the ansatz circuit. Have a look at the implementation of [UCCSD](https://github.com/1QB-Information-Technologies/QEMIST_qSDK/blob/main/qsdk/toolboxes/ansatz_generator/uccsd.py) to see how you can make this more fancy and interesting.

In [12]:
def set_var_params(self, var_params=None):

    if var_params is None:
        var_params = np.random.random(self.n_var_params)
    elif var_params.size != self.n_var_params:
        raise ValueError('Invalid number of parameters.')
    self.var_params = var_params
    return var_params

Next, we'll implement *update_var_params*, where the circuit is updated with a new batch of variational parameters. The *agnostic_simulator* Circuit class keeps a record of the variational gates in the circuit, making this update very straightforward, and avoids having to rebuild the circuit from scratch. All variational gates in the circuit are updated as per the *var_params* argument.

In [57]:
def update_var_params(self, var_params):
    for param_index in range(self.n_var_params):
        self.circuit._variational_gates[param_index].parameter = var_params[param_index]

Next, we'll use the methods from the *qubit_mappings* toolbox to construct a Hartree-Fock reference state. This will just generate a circuit with an X-gate applied to each qubit which we want to begin in the $|1\rangle$ state.

In [7]:
def prepare_reference_state(self):
    circuit = get_reference_circuit(n_spinorbitals=self.n_spinorbitals, n_electrons=self.n_electrons, mapping=self.mapping)
    return circuit

Finally, we'll implement the *build_circuit* method. As compared to the three others here, this is really the only method in the present case that requires much effort--everything else above has followed pretty boilerplate code. For this, we're just going to alternate between entanglers and Euler rotations, using the HEACircuit helper method we defined earlier. We then combine this with the Hartree Fock reference circuit. In the event that no qubits are instantiated as $|1\rangle$, we skip this empty reference circuit.

In [8]:
def build_circuit(self, var_params=None):
    """Construct the variational circuit to be used as our ansatz."""
    self.var_params = self.set_var_params(var_params)

    reference_state_circuit = self.prepare_reference_state()
    hea_circuit = HEACircuit(self.n_qubits, self.n_layers)

    if reference_state_circuit.size != 0:
        self.circuit = reference_state_circuit + hea_circuit
    else:
        self.circuit = hea_circuit
    return self.circuit

# HEA-VQE on an H$_2$-dimer
With the *Ansatz* so defined, we're ready to go ahead and build our VQE solver class, and run a calculation on a molecule of interest. I'm going to use pyscf to build a hydrogen dimer.

In [5]:
from pyscf import gto
H2 = [('H',(0,0,0)),('H',(0,0,0.74137727))]
mol_H2 = gto.Mole()
mol_H2.atom = H2
mol_H2.basis = "sto-3g"
mol_H2.charge = 0
mol_H2.spin = 0
mol_H2.build()

With this molecule prepared, we're ready to instantiate our ansatz, and feed it into VQE. I'll access details of the molecule required to build our ansatz circuit (i.e. number of spin-orbitals and number of electrons) from the molecule object. Feel free to change the number of layers in the circuit, and explore how this changes VQE results, and timing.

In [6]:
n_spinorbitals = 2*mol_H2.nbas
n_electrons = mol_H2.nelectron
hea_layers = 4
HEA_ansatz = HEA(n_spinorbitals=n_spinorbitals, n_electrons=n_electrons, n_layers=hea_layers)

Finally, we can instantiate the *VQESolver*, and run.  Note that we are initializing all variational gates to random values, so convergence may vary from shot to shot.

In [ ]:
vqe_options = {"molecule": mol_H2, "qubit_mapping": 'JW', 'ansatz': HEA_ansatz}

HEA_VQE = VQESolver(vqe_options)
HEA_VQE.build()
HEA_VQE.simulate()

How well did we do here? Let's compare against Hartree Fock and FCI.

In [24]:
energy_fci = HEA_VQE.qemist_molecule.fci_energy
energy_hf = HEA_VQE.qemist_molecule.hf_energy
energy_vqe = HEA_VQE.optimal_energy
print(f'FCI ENERGY: {energy_fci :.7f} Ha')
print(f'HF ENERGY: {energy_hf :.7f} Ha')
print(f'HEA-VQE ENERGY: {energy_vqe :.7f} Ha')

FCI ENERGY: -1.1372704 Ha
HF ENERGY: -1.1166856 Ha
HEA-VQE ENERGY: -1.1371191 Ha


Depending on the random initialization of the variational parameters, we may recover something very close to FCI, or we may get trapped in a local minima and converge somewhere closer to Hartree-Fock. Try running the HEA_VQE code cell a few more times to see how this varies.

In this tutorial, we've seen how to implement a custom ansatz circuit for VQE using the tools from *qSDK*. Hopefully, this gives some impression of how this platform is designed to help users construct their own workflows easily, focusing on the specific issues they are interested in studying without the distraction of building the supporting framework from scratch. 